# This project implemented by Elham Mahdipour
## I am a Ph.D. Graduate  of computer engineering at Yazd University, Yazd, Iran.
## I am a Faculty Member of Computer Engineering Department at Khavaran Institute of Higher Education, Mashhad, Iran.
### Please feel free and contact to me: elham.mahdipour@gmail.com/ e.mahdipour@profs.khi.ac.ir

# Please run all cells for test the RENA model. This program test RENA for HSapiens and DMelanogaster species.

# Phase 1 
## Create Dataset and Preprocessing

In [1]:
import networkx as nx
G1=nx.read_edgelist('biogrid samples\HSapiens.gw')
G1

In [2]:
G2=nx.read_leda('biogrid samples\DMelanogaster.gw') 
G2

In [3]:
### Check and Swap if G1 > G2 ###
print(len(G1))
print(len(G2))
print('*******')
if len(G1)>len(G2):
    temp=G1
    G1=G2
    G2=temp
print(len(G1))
print(len(G2))

13274
7937
*******
7937
13274


In [4]:
G_target_na=nx.read_weighted_edgelist('biogrid samples\DM-HS.tsv')
print(len(G_target_na))
G_target_na  

8118


In [5]:
ed1=G1.edges()
ed2=G2.edges()

nd1=G1.nodes()
nd2=G2.nodes()

el1=list(ed1)
el2=list(ed2)

nd1=list(nd1)
nd2=list(nd2)

degG1 = [val for (node, val) in G1.degree()]
degG2 = [val for (node, val) in G2.degree()]

# Feature Extraction

# compute score for create similarity matrix

In [6]:
def deg_Diff(G1,G2):
    Degree_Difference=np.zeros((len(G1),len(G2)))
    for i in range(len(G1)):
        for j in range(len(G2)):
            Degree_Difference[i][j]=abs(degG1[i]-degG2[j])/max(degG1[i],degG2[j])
    return Degree_Difference

In [7]:
def score_pageRank(X):
    a=nx.pagerank(X)
    return a

In [8]:
def coefficient_pagerank(x,y):  #x is G1, y is G2
   # print(len(x))
    p1=score_pageRank(x)
    b=p1.values()
    pr1=list(b)
    p2=score_pageRank(y)
    c=p2.values()
    pr2=list(c)
    pr=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            #print(pr1[i],pr2[j])
            pr[i][j]=abs(pr1[i]-pr2[j])/max(pr1[i],pr2[j])   #minimum pr is maximum similarity of topology 
    return pr

In [9]:
def coefficient_edges(index_node, G, GraphNumber):
    if GraphNumber==1:
        sum_edge=0        
        for i in G.neighbors(nd1[index_node]):                       
            sum_edge=sum_edge+degG1[nd1.index(i)]
        #print(sum_edge)
        temp=(degG1[index_node]-1) if degG1[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG1[index_node]*temp)
    else:
        sum_edge=0        
        for i in G.neighbors(nd2[index_node]):                       
            sum_edge=sum_edge+degG2[nd2.index(i)]
        #print(sum_edge)
        temp=(degG2[index_node]-1) if degG2[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG2[index_node]*temp)
    return coeff_node

In [10]:
def compute_Ea(G1,G2):
    Ea_G1=np.zeros(len(G1))
    Ea_G2=np.zeros(len(G2))
    for i in range(len(G1)):
        Ea_G1[i]=coefficient_edges(i, G1, 1)
    for j in range(len(G2)):
        Ea_G2[j]=coefficient_edges(j,G2,2)
    ea=[Ea_G1, Ea_G2]
    return(ea)

In [11]:
# compute relative clustering coefficient difference between node a (in G1) and node b (in G2)
def CD(G1, G2):
    cd=np.zeros((len(G1),len(G2)))
    EA=compute_Ea(G1,G2)
    #print(EA[0])      #Ea for G1
    #print("===================")
    #print(EA[1])      #Ea for G2
    for i in range(len(G1)):
        for j in range(len(G2)):
            cd[i,j]=abs(EA[0][i]-EA[1][j])/max(EA[0][i],EA[1][j])
    return cd

In [12]:
elt=G_target_na.edges()
elt=list(elt)
print(len(elt))
elt

4059


[('42128', '11270'),
 ('37142', '11336'),
 ('32594', '23325'),
 ('37028', '51'),
 ('3771960', '60558'),
 ('32769', '93973'),
 ('31347', '64921'),
 ('31256', '8892'),
 ('117361', '2194'),
 ('33923', '57628'),
 ('43102', '3421'),
 ('33798', '8814'),
 ('38565', '3065'),
 ('40999', '3987'),
 ('45757', '9276'),
 ('326250', '145853'),
 ('34132', '7006'),
 ('40495', '27032'),
 ('43005', '201595'),
 ('36400', '56948'),
 ('35535', '680'),
 ('32751', '27338'),
 ('326120', '79885'),
 ('31547', '9718'),
 ('33976', '79892'),
 ('42258', '150'),
 ('326150', '5955'),
 ('32245', '1993'),
 ('41864', '6836'),
 ('38864', '34'),
 ('36319', '23392'),
 ('37217', '37'),
 ('43803', '2047'),
 ('31695', '38'),
 ('35731', '90'),
 ('2768853', '23626'),
 ('326235', '8029'),
 ('33103', '21'),
 ('31880', '30827'),
 ('3885565', '8396'),
 ('45668', '523'),
 ('39160', '8398'),
 ('43483', '8402'),
 ('40656', '65080'),
 ('32947', '65082'),
 ('40451', '6175'),
 ('42803', '3156'),
 ('36956', '86'),
 ('318103', '729991'),
 (

In [13]:
def sequence_score(x,y):
    seq=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            q1=(nd1[i],nd2[j]) in elt
            if q1==False:
                c=0
            else:
               # print(q1)
                c=1
            seq[i][j]=c    
            
    return seq

In [14]:
def compute_score(G1,G2):
    coeff_pr=coefficient_pagerank(G1,G2)
    dd=deg_Diff(G1,G2)
    cd=CD(G1,G2)
    seq_sc=sequence_score(G1,G2)
    
    alpha=0.1
    betta=0.2
    gamma=0.2
    zetta=1-alpha-betta-gamma
    s=alpha*(1-coeff_pr)+betta*(1-dd)+gamma*(1-cd)+zetta*seq_sc
    return s,coeff_pr, dd, cd, seq_sc

In [15]:
import numpy as np
sim, coeff_pr, dd, cd, seq=compute_score(G1,G2)

# Phase 2: Dataset Generation

# create data for deep learning

# change problem to classification 
## [node of G1, node of G2, sequence similarity, Coefficient page rank, clustering coefficient difference,  similarity score, alignment=yes(0)/TN or no(1)/TP]

In [16]:
data=[]
noc=0
yc=0
for i in range(len(nd1)):
    for j in range(len(nd2)):
        if G_target_na.has_edge(nd1[i],nd2[j]):
            align_class='Yes'
            sample=[int(nd1[i]),int(nd2[j]),seq[i][j], coeff_pr[i][j], dd[i][j],cd[i][j],sim[i][j],align_class] 
            data.append(sample)
            yc+=1
        else:
            align_class='No'
            sample=[int(nd1[i]),int(nd2[j]),seq[i][j], coeff_pr[i][j], dd[i][j],cd[i][j],sim[i][j],align_class] 
            data.append(sample)
            noc+=1
print(len(data), yc, noc)

105355738 2652 105353086


In [17]:
from sklearn.utils import shuffle
data = shuffle(data)
X=[]
y=[]
for i in range(len(data)):
    X.append(data[i][0:7])
    y.append(data[i][7])
    
len(X),len(y)

105355738 105355738


In [18]:
X_train=X
y_train=y

# test model for real data

In [19]:
# Label encode Class (Species)
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# One Hot Encode
y_train = np_utils.to_categorical(encoded_Y)

In [20]:
import numpy as np
X_train=np.array(X_train)
y_train=np.array(y_train)

# Phase 3: RENA network

## Load tune RENA model

In [25]:
from keras.models import load_model
#del model
model_rnn = load_model('biogrid samples/deep_model_undersample_RNN_SC-AT.h5')
model_rnn.load_weights('biogrid samples/deep_model_undersample_RNN_weights_SC-AT.h5', by_name=True)

In [26]:
# without resample
result_tr = model_rnn.evaluate(X_train, y_train)
result_tr

105355738/105355738 [==============================] - 3045s 72us/step

[0.0011425101464782, 1.0, 0.001141212194647, 1.3089101761910328e-06]


In [28]:
%%time
out_tr=model_rnn.predict(X_train)
out_tr

# Create alignment file 
f = open("biogrid samples/align-DM-HS.txt", "w")
for i in range(len(out_tr)):
    if out_tr[i][1]>out_tr[i][0]:
        z=[X_train[i][0],X_train[i][1]]
        f.write(str(z))
f.close()

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_train.argmax(axis=1), out_tr.argmax(axis=1)))

from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score
print(precision_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))
print(recall_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))
print(f1_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))

[[105353086        0]
 [       0      2652]]
1.0
1.0
1.0
Wall time: 31min 15s


# test other classifier without resample

In [28]:
x_tra,y_tra=X_train,y_train

In [29]:
X_train,y_train=X,y

In [29]:
%%time
# Linear Discriminant Analysis

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
print('Accuracy of LDA classifier on training set: {:.2f}'
     .format(lda.score(X_train, y_train)))

from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = lda.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

Accuracy of LDA classifier on training set: 1.00
['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999831862598392
[[105353086        0]
 [       2537      115]]
0.9999911321869264
0.7827984650845279
0.86201452842006239
Wall time: 51min 24s


In [30]:
%%time
# KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))

from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = knn.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

Accuracy of KNN classifier on training set: 1.00
['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999621374598216
[[105353086        0]
 [       2652      0]]
0.4999978321669334
0.5
0.49998452952010268
Wall time: 1h 27min 38s


In [32]:
%%time
# SVM
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)
print('Accuracy of SVM classifier on training set: {:.2f}'
     .format(svm.score(X_train, y_train)))

from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = svm.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

Accuracy of SVM classifier on training set: 1.00
['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999731595688063
[[105353086        0]
 [       2652      0]]
0.4999989451697312
0.5
0.49999562953067094
Wall time: 2h 47min 34s
